# Openlimits Python Example

In [1]:
from openlimits_python import ExchangeClient
from dotenv import load_dotenv
import os

## Setup

Initialize a new client from API key data. Exchanges take different initialization params

In [4]:
# this allows python to pull ENV variables from a .env file, nothing to do with the API
load_dotenv()

# exchange initialization and credentials for nash
nash_creds = {
    "nash": {
        "credentials": {
            "nash_credentials": {
                "secret": os.getenv("NASH_API_SECRET"),
                "session": os.getenv("NASH_API_KEY")
            },
        },
        "client_id": 0,
        "environment": "production",
        "affiliate_code": None,
        "timeout": 10000
    }
}

# other exchanges take slightly different initialization credentials
binance_creds = {
    "binance": {
        "credentials": None, # we could provide API keys here
        "sandbox": False,
    }
}

In [5]:
# initialize a Nash client
client = ExchangeClient(nash_creds)

## Account and Market queries

You can find the full set of queries supported [here](https://github.com/nash-io/openlimits-python/blob/master/src/lib.rs)

In [7]:
client.get_account_balances(None)[:5] # None is for pagination + just sample 5

[{'balance': {'asset': 'qnt',
   'free': '0.0011742091125000000000000',
   'total': '0.0011742091125000000000000'}},
 {'balance': {'asset': 'zrx',
   'free': '0.3051298050000000000000000',
   'total': '0.3051298050000000000000000'}},
 {'balance': {'asset': 'nnn',
   'free': '0.0443160562500000000000000',
   'total': '0.0443160562500000000000000'}},
 {'balance': {'asset': 'gunthy',
   'free': '7.9671055753125000000000000',
   'total': '7.9671055753125000000000000'}},
 {'balance': {'asset': 'noia',
   'free': '98.0178965322462500000000000',
   'total': '98.0178965322462500000000000'}}]

In [8]:
client.get_order_history("btc_usdc", None)[:2] # None for pagination + just look at first two

[{'order': {'id': '1606505075762000000',
   'market_pair': 'btc_usdc',
   'price': '16815.90000000',
   'order_type': {'order_type': 'limit'},
   'client_order_id': None,
   'created_at': 1606505075759,
   'side': {'side': 'buy'},
   'size': '0.001064460',
   'status': {'order_status': 'filled'}}},
 {'order': {'id': '1606349176498000000',
   'market_pair': 'btc_usdc',
   'price': '18651.60000000',
   'order_type': {'order_type': 'limit'},
   'client_order_id': None,
   'created_at': 1606349176496,
   'side': {'side': 'buy'},
   'size': '0.001190240',
   'status': {'order_status': 'filled'}}}]

In [4]:
client.limit_buy("eth_usdc", "0.0171", "350.46", {"time_in_force": "1m"})

{'order': {'id': '1604271564576000000',
  'market_pair': 'eth_usdc',
  'price': None,
  'order_type': {'order_type': 'limit'},
  'client_order_id': None,
  'created_at': 1604271564574,
  'side': {'side': 'buy'},
  'size': '0',
  'status': {'order_status': 'pending'}}}

In [9]:
client.get_order("1604271564576000000")

{'order': {'id': '1604271564576000000',
  'market_pair': 'eth_usdc',
  'price': '350.46000000',
  'order_type': {'order_type': 'limit'},
  'client_order_id': None,
  'created_at': 1604271564574,
  'side': {'side': 'buy'},
  'size': '0.017100',
  'status': {'order_status': 'canceled'}}}

In [6]:
client.cancel_order("eth_usdc", "1604271564576000000")

{'order_canceled': '1604271564576000000'}

In [10]:
client.get_historic_trades("btc_usdc", None)[:2] # None is for pagination + just look at first two

[{'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',
   'price': '17785.70000000',
   'qty': '0.001558000',
   'order_id': '1606606221682000000',
   'side': {'side': 'sell'},
   'created_at': 1606606440729,
   'fees': '0',
   'id': 'uZq070kta45yDrn927UzMrvAKg2hj1u-GgyirdWSmC4'}},
 {'trade': {'liquidity': {'liquidity': 'maker'},
   'market_pair': 'btc_usdc',
   'price': '17785.70000000',
   'qty': '0.002655000',
   'order_id': '1606606221564000000',
   'side': {'side': 'sell'},
   'created_at': 1606606440729,
   'fees': '0',
   'id': 'SDqk4VDTrjU_PF56eDX63DF5HNGj3wQiYaB0AVoCMwg'}}]

In [11]:
client.get_ticker("btc_usdc")

{'ticker': {'price': '17785.75000000'}}

## Subscriptions

Subscriptions work by registering a callback function with the client. This callback function will be passed every incoming event on the subscription. The callback executes within a seperate thread managed by the openlimits Rust library. The client instance and other state may be accessed within the callback via declared `global` variables.

In [12]:
state = {"num_events": 0, "events":[]} # a simple example of state

# this will be our callback function
def process_event(event):
    global client # we can use this to access the client and react to events
    global state
    
    state["num_events"] += 1
    state["events"].append(event)

In [13]:
# now we subscribe to the orderbook, passing the callback that we wan to execute
# the second param limits the number of events we get back. we set to 0 as it is not used at the moment
client.subscribe({"orderbook":("btc_usdc", 0)}, process_event) 

In [19]:
# now we wait a few moments and inspect the state
print(state["num_events"])

78


In [20]:
# here is the latest incoming event
print(state["events"][-1])

{'orderbook': {'asks': [], 'bids': [{'ask_or_bid': {'price': '17649.50000000', 'qty': '0.00000000'}}, {'ask_or_bid': {'price': '17649.60000000', 'qty': '1.23394926'}}]}}


## Multiple clients

You can run multiple openlimits clients at the same time. The underlying Rust library will spawn multiple processes with independent runtimes. For example, we can initialize a Binance client simultaneously:

In [21]:
binance_client = ExchangeClient(binance_creds)

We can use it to count the number of available markets on Binance

In [22]:
print(len(binance_client.list_markets()))
print(binance_client.list_markets()[-1])

1193
{'market_pair': {'quote': 'PAX', 'quote_decimal': '0.00001000', 'base': 'ADA', 'base_increment': '0.10000000', 'symbol': 'ADAPAX'}}


Notably, we can use the same API for callbacks here as well. This will only process events from Binance

In [25]:
binance_events = []

def callback_for_binance(event):
    global binance_events
    binance_events.append(event)

In [26]:
# note that market names still differ between exchanges!
binance_client.subscribe({"orderbook": ("btcusdt", 5)}, callback_for_binance)

In [27]:
binance_events[-1]

{'orderbook': {'asks': [{'ask_or_bid': {'price': '17651.39000000',
     'qty': '0.75072900'}},
   {'ask_or_bid': {'price': '17655.12000000', 'qty': '0.26273400'}},
   {'ask_or_bid': {'price': '17655.15000000', 'qty': '0.00000000'}},
   {'ask_or_bid': {'price': '17656.27000000', 'qty': '0.72340700'}},
   {'ask_or_bid': {'price': '17663.14000000', 'qty': '0.00000000'}},
   {'ask_or_bid': {'price': '17688.17000000', 'qty': '0.00000000'}},
   {'ask_or_bid': {'price': '17697.28000000', 'qty': '0.00000000'}},
   {'ask_or_bid': {'price': '18070.92000000', 'qty': '0.00000000'}}],
  'bids': [{'ask_or_bid': {'price': '17651.38000000', 'qty': '2.46288400'}},
   {'ask_or_bid': {'price': '17647.71000000', 'qty': '0.00000000'}},
   {'ask_or_bid': {'price': '17647.64000000', 'qty': '0.01311800'}},
   {'ask_or_bid': {'price': '17647.32000000', 'qty': '0.00000000'}},
   {'ask_or_bid': {'price': '17646.43000000', 'qty': '0.03036200'}},
   {'ask_or_bid': {'price': '17642.22000000', 'qty': '0.00000000'}},